In [1]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [2]:
from datetime import datetime
import os
import pandas as pd
from pycountry import countries

from hopkins_cleaner import HopkinsDataFull
from us_data_cleaner import USDataCleanUp

pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [3]:
self = HopkinsDataFull()
self.run()
df = self.data

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:103: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df["previous_days_date"] = df["date"] + pd.np.timedelta64(1, "D")
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].rank(ascending=True)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [4]:
df.date.max() 

Timestamp('2020-03-21 00:00:00')

In [5]:
df.head(2)

,province_or_state,country_or_region,date,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,running_total_deaths,running_total_deaths_prev_day,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
0,Not Provided,Thailand,2020-01-22,2,Not Provided-Thailand,0.0,2.0,0,0.0,0.0,0,0.0,0.0,NaN,15.0,101.0,NaN,NaN,TH,THA,69428524.0,37.7,2,NaN,NaN
1,Not Provided,Thailand,2020-01-23,3,Not Provided-Thailand,2.0,1.0,0,0.0,0.0,0,0.0,0.0,NaN,15.0,101.0,NaN,NaN,TH,THA,69428524.0,37.7,3,NaN,NaN


In [14]:
df.shape

(17460, 25)

In [16]:
df.drop_duplicates(['province_or_state', 'country_or_region','date']).shape

(17460, 25)

In [22]:
cases = df['daily_new_cases'].sum()

In [23]:
recoveries = df['daily_new_recoveries'].sum()

In [24]:
deaths = df['daily_new_deaths'].sum()

In [28]:
df.date.min(), df.date.max()

(Timestamp('2020-01-22 00:00:00'), Timestamp('2020-03-21 00:00:00'))

In [26]:
cases - (recoveries + deaths)

200052.0

In [17]:
us = df.loc[df['country_code_3'] == 'USA']

In [18]:
us.daily_new_cases.sum()

25489.0

In [ ]:
us.

In [ ]:
pd.__version__

In [ ]:
us.state_name.value_counts()

In [ ]:
df.head(2)

In [ ]:
df.groupby(['country_code_3'])['daily_new_cases'].sum().sort_values()

In [29]:
df.to_csv('../output_data/HOPKINS_CLEANED.csv', index=False)

# QA 

#### Make sure all US states have the same number of days 

#### Total US cases 

In [30]:
us['daily_new_cases'].sum(), us['daily_new_recoveries'].sum(), us['daily_new_deaths'].sum()

(25489.0, 0.0, 307.0)

In [31]:
us.groupby(['date'])['daily_new_cases'].sum().reset_index().head(5)

,date,daily_new_cases
0,2020-01-22,1.0
1,2020-01-23,0.0
2,2020-01-24,1.0
3,2020-01-25,0.0
4,2020-01-26,3.0


## Check that daily counts are working 

In [ ]:
test_daily_vs_total_logic(df)

In [ ]:
df.loc[df['province_or_state'] == 'WA'].tail(5)

In [ ]:
df.loc[df['province_or_state'] == 'VA'].tail(5)

In [ ]:
df.loc[df['province_or_state'] == 'CA'].tail(5)

In [ ]:
df.loc[df['country_or_region'].str.contains('Korea')].tail(5)

In [ ]:
yesterday.sort_values('running_total_cases', ascending=False)[['country_or_region','running_total_cases','running_total_recoveries','running_total_deaths','country_population_2018','country_median_age']]

## QA of Data 

In [32]:
# This indicates a shift in the data
df.loc[df['daily_new_cases'] < 0]['date'].value_counts()

2020-03-16    4
2020-03-18    4
2020-03-06    2
2020-03-13    1
2020-01-23    1
2020-02-29    1
2020-02-07    1
2020-03-09    1
2020-01-31    1
2020-03-20    1
2020-02-02    1
Name: date, dtype: int64

In [33]:
# This indicates a shift in the data
us.loc[us['daily_new_cases'] < 0]

,province_or_state,country_or_region,date,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,running_total_deaths,running_total_deaths_prev_day,daily_new_deaths,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg,hundred_case_state_rank,hundred_case_country_rank
16938,Grand Princess,US,2020-03-16,20,Grand Princess-US,21.0,-1.0,0,0.0,0.0,0,0.0,0.0,Grand Princess,NaN,NaN,10.0,43.0,US,USA,327167434.0,38.1,4632,NaN,10.0
17228,NV,US,2020-03-18,55,NV-US,56.0,-1.0,0,0.0,0.0,1,1.0,0.0,Nevada,39.876019,-117.224121,8.0,45.0,US,USA,327167434.0,38.1,7783,NaN,12.0
17374,UT,US,2020-03-20,78,UT-US,80.0,-2.0,0,0.0,0.0,0,0.0,0.0,Utah,39.419220,-111.950684,9.0,47.0,US,USA,327167434.0,38.1,19100,NaN,14.0
17420,WA,US,2020-03-18,1014,WA-US,1076.0,-62.0,0,1.0,-1.0,55,55.0,0.0,Washington,47.751076,-120.740135,19.0,45.0,US,USA,327167434.0,38.1,7783,12.0,12.0


In [34]:
us['daily_new_cases'].sum(), us['daily_new_recoveries'].sum(), us['daily_new_deaths'].sum()

(25489.0, 0.0, 307.0)

In [35]:
# This indicates a shift in the data
df.loc[df['daily_new_cases'] < 0]['province_or_state'].value_counts()

Not Provided             7
Queensland               2
NV                       1
WA                       1
UT                       1
Northern Territory       1
Saint Barthelemy         1
Guizhou                  1
Diamond Princess         1
Grand Princess           1
From Diamond Princess    1
Name: province_or_state, dtype: int64